### Output the dispersion curves for overplotting on wavenumber-frequency spectra

In [9]:
pathout="/project/cas/islas/python_savs/CAM7_vertres_paper/DATA_SORT/dispersion_curves/"

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from CASutils import plotting_utils as myplots

### Kelvin waves

Dispersion relation: $\omega = \sqrt{gh_{e}}k

Phase speed: $c = \sqrt{gh_e}$

- $\omega$ = angular frequency in rad s$^{-1}$
- $g$ = acceleration due to gravity (9.81 ms$^{-1}$)
- $h_{e}$ = equivalent depth in m
- $k$ = wavenumber in rad m$^{-1}$

In [2]:
a = 6.371e6
g = 9.81
h_e = [12, 25, 50] # equivalent depths following Wheeler and Kiladis
k = np.arange(-14,14+1,1)
k_rad_per_m = k * 2 * np.pi / (2 * np.pi * a)

# - loop over equivalent depths
w_kelvin = []
for i in np.arange(0,len(h_e),1):
    D = h_e[i] # equivalent depth
    w = np.sqrt(g*D)*k_rad_per_m
    w = w*86400./(2*np.pi) # convert from rad/s to cycles/day
    w = xr.DataArray(w, dims=['k'], coords=[k], name='w_kelvin')
    w_kelvin.append(w)

w_kelvin = xr.concat(w_kelvin, dim='h_e')
w_kelvin['h_e'] = h_e

### Inertio-gravity waves

Dispersion relation: $\omega^{2} = (2n + 1)\beta  \sqrt{gh_e} + gh_{e}k^{2}$ (Gill, equion 11.6.7)

Phase speed: c=$\sqrt{g h_{e}}$

- $\omega$ = angular frequency (rad/s)
- $\beta$ beta parameter $2\Omega cos(\phi_{o})/a$
- $g$ = acceleration due to gravity (9.81 ms$^{-1}$)
- $h_{e}$ = equivalent depth
- $k$ = wavenumber in rad m$^{-1}$
- $n$ is the order of the parabolic cylinder function

In [12]:
a = 6.371e6
g = 9.81
h_e = [12, 25, 50] # equivalent depths following Wheeler and Kiladis
k = np.arange(-14,14+1,1)
k_rad_per_m = k * 2 * np.pi / (2 * np.pi * a)
beta=(2/a) * (2*np.pi / (24.*60.*60.)) *np.cos(0)

#----n=1
n=1

# - loop over equivalent depths
w_ig_n1 = []
for i in np.arange(0,len(h_e),1):
    D = h_e[i] # equivalent depth
    w2= (2*n + 1)*beta*np.sqrt(g*D) + g*D*(k_rad_per_m**2.)
    w = np.sqrt(w2)
    w = w*86400./(2*np.pi) # convert from rad/s to cycles/day
    w = xr.DataArray(w, dims=['k'], coords=[k], name='w_ig_n1')
    w_ig_n1.append(w)
    
w_ig_n1 = xr.concat(w_ig_n1, dim='h_e')
w_ig_n1['h_e'] = h_e


#----n=2
n=2

# - loop over equivalent depths
w_ig_n2 = []
for i in np.arange(0,len(h_e),1):
    D = h_e[i] # equivalent depth
    w2= (2*n + 1)*beta*np.sqrt(g*D) + g*D*(k_rad_per_m**2.)
    w = np.sqrt(w2)
    w = w*86400./(2*np.pi) # convert from rad/s to cycles/day
    w = xr.DataArray(w, dims=['k'], coords=[k], name='w_ig_n2')
    w_ig_n2.append(w)
    
w_ig_n2 = xr.concat(w_ig_n2, dim='h_e')
w_ig_n2['h_e'] = h_e

### Equatorially trapped Rossby waves

Dispersion relation: $\omega = - \frac{\beta k}{k^{2} + \frac{1}{\sqrt{gh_{e}}}(2n + 1)\beta}$

- $\omega$ = angular frequency (rad/s)
- $\beta$ = beta parameter $2\Omega cos(\phi_{o}) / a$
- $k$ = zonal wavenumber
- $g$ = acceleration due to gravity (9.81m/s)
- $h_{e}$ = equivalent depth
- $n$ is the order of the parabolic cylinder function

In [4]:
a = 6.371e6
g = 9.81
h_e = [ 12, 25, 50]
k = np.arange(-14,14+1,1)
k_rad_per_m = k*2*np.pi / (2*np.pi*a)
beta = (2/a) * (2*np.pi / (24.*60.*60.))*np.cos(0)
n=1

# - loop over equivalent depths
w_etrap = []
for i in np.arange(0,len(h_e),1):
    D = h_e[i] # equivalent depth
    w = -1.*(beta*k_rad_per_m) / ( k_rad_per_m**2 + (1/np.sqrt(g*D))*(2*n + 1)*beta)
    #w = -1.*(beta*k_rad_per_m) / ( k_rad_per_m**2 + (1/(g*D))*(2*n + 1)*beta)
    w = w*86400./(2*np.pi) # convert from rad/s to cycles/day
    w = xr.DataArray(w, dims=['k'], coords=[k], name='w_etrap')
    w_etrap.append(w)
w_etrap = xr.concat(w_etrap, dim='h_e')
w_etrap['h_e'] = h_e

### Mixed Rossby-Gravity Waves

Dispersion relation: $\frac{\omega}{c} - k - \frac{\beta}{\omega} = 0$ (Gill equation 11.6.9).

This has roots:

$\frac{ kc \pm \sqrt{k^{2}c^{2} + 4\beta c}}{2}$

Only using the positive one.  $c=\sqrt{gh_{e}}$.

- $\omega$ = the angular frequency (in rad/s)
- $\beta$ = the beta parameter $2\Omega cos(\phi_{o}) / a$
- $g$ = the acceleration due to gravity (9.81m/s)
- $h_{e}$ is the equivalent depth
- $k$ is the horizontal wavenumber (in rad/m)
- $a$ = the Earth's radius

In [6]:
a = 6.371e6
g = 9.81
h_e = [ 12, 25, 50]
k = np.arange(-14,14+1,1)
k_rad_per_m = k*2*np.pi / (2*np.pi*a)
beta = (2/a) * (2*np.pi / (24.*60.*60.))*np.cos(0)

w_mrg=[]

for i in np.arange(0,len(h_e),1):
    D = h_e[i] # equivalent depth
    term1 = (1./2.)*k_rad_per_m * np.sqrt(g*D)
    term2 = (1./2.)*np.sqrt( ((k_rad_per_m*np.sqrt(g*D))**2) + (4*beta*np.sqrt(g*D)) )
    w = term1 + term2
    w = w*86400./(2*np.pi)
    w = xr.DataArray(w, dims=['k'], coords=[k], name='w_mrg')
    w_mrg.append(w)
w_mrg = xr.concat(w_mrg, dim='h_e')
w_mrg['h_e'] = h_e

In [14]:
datout = xr.merge([w_kelvin, w_ig_n1, w_ig_n2, w_etrap, w_mrg])
datout.to_netcdf(pathout+'dispersion_curves.nc')